# **Keyphrase Extraction using BERT Embeddings Method**

In [ ]:
!pip install transformers torch seqeval sklearn datasets
!pip install seqeval
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=17e78c4fb5cac6ee03cc24663d00d546f542abdda85ceb8ba5d9b87a19614008
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from collections import Counter
import re
import torch
from torch.nn import CrossEntropyLoss

# Download data
dataset = load_dataset("midas/kp20k", split="train").select(range(5000))  # استخدام 5000 عينة

# Convert BIO labels to numbers
BIO_MAP = {"O": 0, "B": 1, "I": 2}

# Text cleaning function
def clean_text(text):
    if isinstance(text, list):  # If the text is a list of words
        text = " ".join(text)
    # Remove symbols other than letters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Divide texts into words
    tokens = text.split()
    return tokens

# Text processing and BIO nomenclature
def preprocess_data(example):
    # تنظيف النصوص
    tokens = clean_text(example['document'])
    tags = example['doc_bio_tags']

    # Make sure the length matches the words and labels
    max_length = 512
    tokens = tokens[:max_length]
    tags = tags[:max_length]

    # Convert labels to numbers
    tags = [BIO_MAP[tag] for tag in tags]

    return {"tokens": tokens, "tags": tags}

# Data processing application
processed_dataset = dataset.map(preprocess_data)

# Oversampling for classes "B" and "I"
def oversample_data(dataset):
    oversampled_data = []
    for example in dataset:
        tags = example["tags"]
        tag_counts = Counter(tags)

        # If it contains a "B" or an "I," repeat it several times
        if tag_counts[BIO_MAP["B"]] > 0 or tag_counts[BIO_MAP["I"]] > 0:
            oversampled_data.extend([example] * 3)  # تكرار 3 مرات
        else:
            oversampled_data.append(example)

    return oversampled_data

oversampled_dataset = oversample_data(processed_dataset)
balanced_dataset = Dataset.from_list(oversampled_dataset)

# Download Tokenizer for SciBERT model
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Convert text and labels to input format for the form
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=512
    )
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word IDs
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignore these codes in the calculation
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the conversion function
tokenized_dataset = balanced_dataset.map(tokenize_and_align_labels, batched=True)

# Split data into training and testing
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Download the SciBERT model with classifications setup
model = AutoModelForTokenClassification.from_pretrained(
    "allenai/scibert_scivocab_uncased",
    num_labels=3,  # عدد الفئات: B, I, O
    id2label={0: "O", 1: "B", 2: "I"},
    label2id={"O": 0, "B": 1, "I": 2}
)

# **Gradual Unfreezing **
# Freezing the first layers (8 layers)
for param in model.bert.encoder.layer[:8].parameters():
    param.requires_grad = False

# Trainer preparation
data_collator = DataCollatorForTokenClassification(tokenizer)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Initial training
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# First stage training
trainer.train()

# ** Unfreeze extra layers (Gradual Unfreezing) **
# Unfreeze the next layers (layers 8-12)
for param in model.bert.encoder.layer[:12].parameters():
    param.requires_grad = True

# Update training settings
training_args.num_train_epochs = 5  # Additional training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# Second stage training
trainer.train()

# Model evaluation
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

# Restore text and labels while discarding filled symbols and reconverting numbers to text labels
reverse_BIO_MAP = {0: "O", 1: "B", 2: "I"}
true_labels = []
predicted_labels = []

for label_list, prediction_list in zip(labels, predictions):
    filtered_true_labels = []
    filtered_predicted_labels = []
    for label, prediction in zip(label_list, prediction_list):
        if label != -100:  # Ignore filled icons
            filtered_true_labels.append(reverse_BIO_MAP[label])
            filtered_predicted_labels.append(reverse_BIO_MAP[prediction])
    true_labels.append(filtered_true_labels)
    predicted_labels.append(filtered_predicted_labels)

# Performance calculation
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("Accuracy:", accuracy_score(true_labels, predicted_labels))
print("Precision:", precision_score(true_labels, predicted_labels))
print("Recall:", recall_score(true_labels, predicted_labels))
print("F1 Score:", f1_score(true_labels, predicted_labels))
print("Classification Report:\n", classification_report(true_labels, predicted_labels))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.63k [00:00<?, ?B/s]

kp20k.py:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


test.jsonl:   0%|          | 0.00/51.6M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.153700,0.152137
2,0.129600,0.130963
3,0.126300,0.126907


Epoch,Training Loss,Validation Loss
1,0.082100,0.095002
2,0.047300,0.060763
3,0.035800,0.044422


Epoch,Training Loss,Validation Loss
1,0.082100,0.095002
2,0.047300,0.060763
3,0.035800,0.044422
4,0.014000,0.038900
5,0.011600,0.036633


Accuracy: 0.9908405334096623
Precision: 0.8444717780511138
Recall: 0.9032305218535301
F1 Score: 0.8728633987851391
Classification Report:
               precision    recall  f1-score   support

           _       0.84      0.90      0.87     13682

   micro avg       0.84      0.90      0.87     13682
   macro avg       0.84      0.90      0.87     13682
weighted avg       0.84      0.90      0.87     13682



# **Keyphrase Extraction using Traditional Method**

In [ ]:
!pip install transformers torch seqeval sklearn datasets
!pip install seqeval
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d731fec143ddd9bf014bad4caaa0b8d722f4c97ebcdac422c0a6e0b68e51d2ad
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from collections import Counter
import re
import torch
from torch.nn import CrossEntropyLoss

# Download data
dataset = load_dataset("midas/kp20k", split="train").select(range(5000))  # استخدام 5000 عينة

# Convert BIO labels to numbers
BIO_MAP = {"O": 0, "B": 1, "I": 2}

# Text cleaning function
def clean_text(text):
    if isinstance(text, list):  # If the text is a list of words
        text = " ".join(text)
    # Remove symbols other than letters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Divide texts into words
    tokens = text.split()
    return tokens

# Text processing and BIO nomenclature
def preprocess_data(example):
    # تنظيف النصوص
    tokens = clean_text(example['document'])
    tags = example['doc_bio_tags']

    # Make sure the length matches the words and labels
    max_length = 512
    tokens = tokens[:max_length]
    tags = tags[:max_length]

    # Convert labels to numbers
    tags = [BIO_MAP[tag] for tag in tags]

    return {"tokens": tokens, "tags": tags}

# Data processing application
processed_dataset = dataset.map(preprocess_data)

# Oversampling for classes "B" and "I"
def oversample_data(dataset):
    oversampled_data = []
    for example in dataset:
        tags = example["tags"]
        tag_counts = Counter(tags)

        # If it contains a "B" or an "I," repeat it several times
        if tag_counts[BIO_MAP["B"]] > 0 or tag_counts[BIO_MAP["I"]] > 0:
            oversampled_data.extend([example] * 3)  # Repeat 3 times
        else:
            oversampled_data.append(example)

    return oversampled_data

oversampled_dataset = oversample_data(processed_dataset)
balanced_dataset = Dataset.from_list(oversampled_dataset)

# Download Tokenizer for SciBERT model
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Convert text and labels to input format for the form
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=512
    )
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word identifiers
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignore these codes in the calculation
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the conversion function
tokenized_dataset = balanced_dataset.map(tokenize_and_align_labels, batched=True)

# Split data into training and testing
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Download the SciBERT model with classifications setup
model = AutoModelForTokenClassification.from_pretrained(
    "allenai/scibert_scivocab_uncased",
    num_labels=3,  # عدد الفئات: B, I, O
    id2label={0: "O", 1: "B", 2: "I"},
    label2id={"O": 0, "B": 1, "I": 2}
)

# ** Freeze the lower layers (Gradual Unfreezing) **
# Freezing the first layers (8 layers)
for param in model.bert.encoder.layer[:8].parameters():
    param.requires_grad = False

# Trainer preparation (Trainer)
data_collator = DataCollatorForTokenClassification(tokenizer)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Initial training
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# First stage training
trainer.train()

# ** Unfreeze extra layers (Gradual Unfreezing) **
# Unfreeze the next layers (layers 8-12)
for param in model.bert.encoder.layer[:12].parameters():
    param.requires_grad = True

# Update training settings
training_args.num_train_epochs = 5  # Additional training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# Second stage training
trainer.train()

# Model evaluation
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

# Restore text and labels while discarding filled symbols and reconverting numbers to text labels
reverse_BIO_MAP = {0: "O", 1: "B", 2: "I"}
true_labels = []
predicted_labels = []

for label_list, prediction_list in zip(labels, predictions):
    filtered_true_labels = []
    filtered_predicted_labels = []
    for label, prediction in zip(label_list, prediction_list):
        if label != -100:  # تجاهل الرموز المعبأة
            filtered_true_labels.append(reverse_BIO_MAP[label])
            filtered_predicted_labels.append(reverse_BIO_MAP[prediction])
    true_labels.append(filtered_true_labels)
    predicted_labels.append(filtered_predicted_labels)

# Performance calculation
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("Accuracy:", accuracy_score(true_labels, predicted_labels))
print("Precision:", precision_score(true_labels, predicted_labels))
print("Recall:", recall_score(true_labels, predicted_labels))
print("F1 Score:", f1_score(true_labels, predicted_labels))
print("Classification Report:\n", classification_report(true_labels, predicted_labels))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.63k [00:00<?, ?B/s]

kp20k.py:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


test.jsonl:   0%|          | 0.00/51.6M [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.143700,0.146108
2,0.112700,0.136016
3,0.102100,0.130053


Epoch,Training Loss,Validation Loss
1,0.082600,0.093281
2,0.046800,0.071336
3,0.027200,0.055528
4,0.016800,0.045713
5,0.015400,0.044336


Accuracy: 0.9898644819714069
Precision: 0.8407020364415863
Recall: 0.8919053372183925
F1 Score: 0.8655470878306148
Classification Report:
               precision    recall  f1-score   support

           _       0.84      0.89      0.87     14071

   micro avg       0.84      0.89      0.87     14071
   macro avg       0.84      0.89      0.87     14071
weighted avg       0.84      0.89      0.87     14071

